In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [26]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f5fa52abd8ed451a8c16c01f26c9ddca = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='u7gV-G1eFB5c4dCf6Um1nutSBE7UL5Wgg7xaYIsDwwXs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_f5fa52abd8ed451a8c16c01f26c9ddca.get_object(Bucket='usedcarresaleprice-donotdelete-pr-rw4ipynhvpj2ch',Key='autos.xlsx')['Body']

df = pd.read_excel(body.read())
df.head()


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480.0,test,NaN,1993.0,manuell,0.0,golf,150000,0.0,benzin,volkswagen,NaN,2016-03-24,0.0,70435.0,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300.0,test,coupe,2011.0,manuell,190.0,NaN,125000,5.0,diesel,audi,ja,2016-03-24,0.0,66954.0,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800.0,test,suv,2004.0,automatik,163.0,grand,125000,8.0,diesel,jeep,NaN,2016-03-14,0.0,90480.0,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500.0,test,kleinwagen,2001.0,manuell,75.0,golf,150000,6.0,benzin,volkswagen,nein,2016-03-17,0.0,91074.0,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600.0,test,kleinwagen,2008.0,manuell,69.0,fabia,90000,7.0,diesel,skoda,nein,2016-03-31,0.0,60437.0,2016-04-06 10:17:21


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371539 entries, 0 to 371538
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   dateCrawled          371539 non-null  object        
 1   name                 371524 non-null  object        
 2   seller               371538 non-null  object        
 3   offerType            371538 non-null  object        
 4   price                371538 non-null  float64       
 5   abtest               371538 non-null  object        
 6   vehicleType          333669 non-null  object        
 7   yearOfRegistration   371537 non-null  float64       
 8   gearbox              351329 non-null  object        
 9   powerPS              371538 non-null  float64       
 10  model                351054 non-null  object        
 11  kilometer            371538 non-null  object        
 12  monthOfRegistration  371537 non-null  float64       
 13  fuelType      

In [28]:
df.shape

(371539, 20)

In [29]:
df.isnull().sum()

dateCrawled                0
name                      15
seller                     1
offerType                  1
price                      1
abtest                     1
vehicleType            37870
yearOfRegistration         2
gearbox                20210
powerPS                    1
model                  20485
kilometer                  1
monthOfRegistration        2
fuelType               33388
brand                      2
notRepairedDamage      72062
dateCreated                2
nrOfPictures               2
postalCode                 2
lastSeen                   2
dtype: int64

In [30]:
df=df.drop(['offerType','seller'],axis=1)

In [31]:
#Cars having power less than 50ps and above 900ps seems a little suspicious,
#let's remove them and see what we've got now
df = df[(df.powerPS > 50) & (df.powerPS < 900)]
print(df.shape)
#around 50000 cars ahave been removed which could have inrouduced error to our data

(319717, 18)


In [32]:
#simlarly, filtering our the cars having registeration years not in the mentioned range
#print(df.shape)
df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]
print(df.shape)
# not much of a difference but still, 10000 rows have been reduced. it's better to 
#get rid of faulty data instead of keeping them just to increase the size.

(309179, 18)


In [33]:
#removing irrelevant columns which are either the same for all the cars in teh dataset, or can
#introduce bias, so removing them too.
df.drop(['name','abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen',
         'postalCode', 'dateCreated'], axis='columns', inplace=True)

In [34]:
#dropping the duplicates from the dataframe and stroing it in a new df.
#here all rows having same value in all the mentioned columns will be deleted and by default,
#only first occurance of anysuch row is kept
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage'])

#after removing duplicates
print(new_df.shape)

(285151, 11)


In [35]:
#As the dataset contained some german words for many features, cahnging them to english
new_df.gearbox.replace(('manuell', 'automatik'), ('manual','automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio','kombi','andere'),
                           ('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [36]:
#### Removing the outliers
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]

In [37]:
#Filling NaN values for columns whose data might not be there with the information provider,
#which might lead to some variance but our model
#but we will still be able to give some estimate to the user
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [38]:
new_df.isnull().sum()

price                  0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

In [39]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278583 entries, 1 to 371538
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   price                278583 non-null  float64
 1   vehicleType          278583 non-null  object 
 2   yearOfRegistration   278583 non-null  float64
 3   gearbox              278583 non-null  object 
 4   powerPS              278583 non-null  float64
 5   model                278583 non-null  object 
 6   kilometer            278583 non-null  object 
 7   monthOfRegistration  278583 non-null  float64
 8   fuelType             278583 non-null  object 
 9   brand                278583 non-null  object 
 10  notRepairedDamage    278583 non-null  object 
dtypes: float64(4), object(7)
memory usage: 25.5+ MB


In [40]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300.0,coupe,2011.0,manual,190.0,not-declared,125000,5.0,diesel,audi,Yes
2,9800.0,suv,2004.0,automatic,163.0,grand,125000,8.0,diesel,jeep,not-declared
3,1500.0,small car,2001.0,manual,75.0,golf,150000,6.0,petrol,volkswagen,No
4,3600.0,small car,2008.0,manual,69.0,fabia,90000,7.0,diesel,skoda,No
5,650.0,limousine,1995.0,manual,102.0,3er,150000,10.0,petrol,bmw,Yes


In [42]:
lb=LabelEncoder()
new_df['vehicleType']=lb.fit_transform(new_df['vehicleType'])

In [43]:
new_df['gearbox']=lb.fit_transform(new_df['gearbox'])

In [51]:
new_df['model']=new_df['model'].astype('str')

In [52]:
new_df['model']=lb.fit_transform(new_df['model'])

In [45]:
new_df['fuelType']=lb.fit_transform(new_df['fuelType'])

In [46]:
new_df['brand']=lb.fit_transform(new_df['brand'])

In [47]:
new_df['notRepairedDamage']=lb.fit_transform(new_df['notRepairedDamage'])

In [53]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300.0,3,2011.0,1,190.0,162,125000,5.0,1,1,1
2,9800.0,8,2004.0,0,163.0,118,125000,8.0,1,14,2
3,1500.0,7,2001.0,1,75.0,117,150000,6.0,7,38,0
4,3600.0,7,2008.0,1,69.0,102,90000,7.0,1,31,0
5,650.0,4,1995.0,1,102.0,11,150000,10.0,7,2,1


In [54]:
x = new_df.drop('price',axis=1)
y = new_df['price']

In [55]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 10)

In [56]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train)
ypred=(rf.predict(x_test))
score=r2_score(y_test,ypred)
print('***Random Forest Regressor Model***')
print('Score for Random Forest Regressor Model is {}'.format(score))

***Random Forest Regressor Model***
Score for Random Forest Regressor Model is 0.8676474900795378


In [57]:
pickle.dump(rf,open("model.pkl","wb"))

In [58]:
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 1.7 MB 25.5 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.189
    Uninstalling ibm-watson-machine-learning-1.0.189:
      Successfully uninstalled ibm-watson-machine-learning-1.0.189


In [60]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [59]:
wml_credentials = {
    "apikey":"NQXwjISU0autKm_YBa3KLuKZAdU9kRq1o_0vclfa-xTb",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [62]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.
Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------------  ------------------------
ID                                    NAME                CREATED
ae995cae-e840-4078-82ff-a0d741f6c1fb  usedCarResalePrice  2022-03-25T11:39:56.803Z
------------------------------------  ------------------  ------------------------


In [63]:
SPACE_ID="ae995cae-e840-4078-82ff-a0d741f6c1fb"

In [64]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [65]:
wml_client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a17-24d8-5082-900f-0ab31fbfd3cb  base
scikit-learn_0.22-py3.6        154010f

In [66]:
MODEL_NAME = 'UsedCarModel1'
DEPLOYMENT_NAME = 'used_car_deploy_3.8'
Life_MODEL = rf

In [67]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('default_py3.8')

# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [ ]:
#Save model
model_details = wml_client.repository.store_model(
    model=Life_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [ ]:
model_details

In [ ]:
model_uid = wml_client.repository.get_model_id(model_details)

In [ ]:
model_uid

In [ ]:
wml_client.connections.list_datasource_types()

In [ ]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
# Deploy
deployment1 = wml_client.deployments.create(
    artifact_uid=model_uid, 
    meta_props=deployment_props 
)